In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import re
import string

In [2]:
df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")


In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_fake["Label"] = "fake"
df_real["Label"] = "real"

In [6]:
df = pd.concat([df_fake , df_real])
df.head()

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


In [7]:
df =df.drop(["title", "subject", "date"] , axis = 1)

In [8]:
df = df.dropna()

In [9]:
temp1 = pd.read_csv("train.csv")
temp1.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
temp1 = temp1.rename(columns={'label':'Label'})

In [11]:
conversion_dict = {0 : "real" , 1 : "fake"}
temp1['Label'] = temp1['Label'].replace(conversion_dict)

In [12]:
temp1 = temp1.drop(['id' , 'title' , 'author'] ,axis = 1)
temp1.head()

,text,Label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,fake
1,Ever get the feeling your life circles the rou...,real
2,"Why the Truth Might Get You Fired October 29, ...",fake
3,Videos 15 Civilians Killed In Single US Airstr...,fake
4,Print \nAn Iranian woman has been sentenced to...,fake


In [13]:
df = pd.concat([df,temp1])

In [14]:
df = df.sample(frac = 1)
df.shape

(65698, 2)

In [15]:
df = df.dropna()
df.shape
df.head()

,text,Label
7403,GENEVA (AFP) — European countries must stop...,real
8046,Man Wearing ‘Jewmerica’ T-Shirt Never Dreamed ...,fake
14017,Did Target really believe that 99.8% of Americ...,fake
19052,"On Thursday, MSNBC host Katy Tur seemed to con...",real
19812,Media and Hollywood elites were among the many...,real


In [16]:
df.reset_index(inplace = True)


In [17]:
def Processing(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [18]:
df['text'] = df['text'].apply(Processing)
X = df['text']
Y = df['Label']

In [19]:
df.head()

,index,text,Label
0,7403,geneva afp european countries must stop...,real
1,8046,man wearing jewmerica t shirt never dreamed ...,fake
2,14017,did target really believe that of americans...,fake
3,19052,on thursday msnbc host katy tur seemed to con...,real
4,19812,media and hollywood elites were among the many...,real


In [20]:
df.drop(['index'] , axis = 1 , inplace = True)
df.head()

,text,Label
0,geneva afp european countries must stop...,real
1,man wearing jewmerica t shirt never dreamed ...,fake
2,did target really believe that of americans...,fake
3,on thursday msnbc host katy tur seemed to con...,real
4,media and hollywood elites were among the many...,real


In [21]:
x_train, x_test , y_train , y_test  =  train_test_split(X, Y, test_size = 0.25)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x_train)
Xtest = vectorizer.transform(x_test)

## Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
LR_model = LogisticRegression()
LR_model.fit(x , y_train)
pred = LR_model.predict(Xtest)
print("Logistic Regression  : " ,LR_model.score(Xtest , y_test))
print(classification_report(y_test , pred))

Logistic Regression  :  0.9516296070667073
              precision    recall  f1-score   support

        fake       0.95      0.96      0.95      8459
        real       0.96      0.94      0.95      7956

    accuracy                           0.95     16415
   macro avg       0.95      0.95      0.95     16415
weighted avg       0.95      0.95      0.95     16415



In [27]:
## Logistic Regression
True_Negative, False_Positive, False_Negative, True_Positive = result(y_test , pred)
ACC = (True_Positive+True_Negative)/(True_Positive+False_Positive+False_Negative+True_Negative)*100
print('Logistic Regression')
print(f'Accuracy :{ACC}')
print(f'True Negattive: {(True_Negative/(True_Negative+False_Positive))*100}')
print(f'False Positive : {(False_Positive/(False_Positive+True_Negative))*100}')
print(f'False Negative : {(False_Negative/(True_Positive+False_Negative))*100}')
print(f'True Positive: {(True_Positive/(True_Positive+False_Negative))*100}')
    

Logistic Regression
Accuracy :95.16296070667073
True Negattive: 96.07518619222131
False Positive : 3.924813807778697
False Negative : 5.806938159879336
True Positive: 94.19306184012066


## Decision Tree Classification

In [28]:
from  sklearn.tree import DecisionTreeClassifier

In [29]:
DTC_model = DecisionTreeClassifier()
DTC_model.fit(x , y_train)
pred = DTC_model.predict(Xtest)
print("Decision Tree Classification :" , DTC_model.score(Xtest , y_test))

print(classification_report(y_test , pred))

Decision Tree Classification : 0.9340846786475784
              precision    recall  f1-score   support

        fake       0.93      0.94      0.94      8459
        real       0.94      0.93      0.93      7956

    accuracy                           0.93     16415
   macro avg       0.93      0.93      0.93     16415
weighted avg       0.93      0.93      0.93     16415



In [30]:
## Decision Tree Classification
True_Negative, False_Positive, False_Negative, True_Positive = result(y_test , pred)
ACC = (True_Positive+True_Negative)/(True_Positive+False_Positive+False_Negative+True_Negative)*100
print('Decision Tree Classification')
print(f'Accuracy :{ACC}')
print(f'True Negattive: {(True_Negative/(True_Negative+False_Positive))*100}')
print(f'False Positive : {(False_Positive/(False_Positive+True_Negative))*100}')
print(f'False Negative : {(False_Negative/(True_Positive+False_Negative))*100}')
print(f'True Positive: {(True_Positive/(True_Positive+False_Negative))*100}')

Decision Tree Classification
Accuracy :93.40846786475784
True Negattive: 94.19553138668874
False Positive : 5.804468613311266
False Negative : 7.428355957767723
True Positive: 92.57164404223228


## Passive Aggressive Classifier

In [31]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [32]:
PAC_model = PassiveAggressiveClassifier()
PAC_model.fit(x , y_train)
pred = PAC_model.predict(Xtest)
print("Passive Aggressive Classifier :" , PAC_model.score(Xtest , y_test))
print(classification_report(y_test , pred))

Passive Aggressive Classifier : 0.9679561376789522
              precision    recall  f1-score   support

        fake       0.97      0.97      0.97      8459
        real       0.97      0.96      0.97      7956

    accuracy                           0.97     16415
   macro avg       0.97      0.97      0.97     16415
weighted avg       0.97      0.97      0.97     16415



In [33]:
## Passive Aggressive Classifier
True_Negative, False_Positive, False_Negative, True_Positive = result(y_test , pred)
ACC = (True_Positive+True_Negative)/(True_Positive+False_Positive+False_Negative+True_Negative)*100
print('Passive Aggressive Classifier')
print(f'Accuracy :{ACC}')
print(f'True Negattive: {(True_Negative/(True_Negative+False_Positive))*100}')
print(f'False Positive : {(False_Positive/(False_Positive+True_Negative))*100}')
print(f'False Negative : {(False_Negative/(True_Positive+False_Negative))*100}')
print(f'True Positive: {(True_Positive/(True_Positive+False_Negative))*100}')


Passive Aggressive Classifier
Accuracy :96.79561376789522
True Negattive: 97.26918075422627
False Positive : 2.7308192457737324
False Negative : 3.707893413775767
True Positive: 96.29210658622424


## Gradient Boosting Classifier

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
GBC_model = GradientBoostingClassifier(random_state = 0)
GBC_model.fit(x , y_train)
pred = GBC_model.predict(Xtest)
print("Gradient Boosting Classifier :" , GBC_model.score(Xtest , y_test))
print(classification_report(y_test , pred))


Gradient Boosting Classifier : 0.9495583307950046
              precision    recall  f1-score   support

        fake       0.93      0.97      0.95      8459
        real       0.97      0.93      0.95      7956

    accuracy                           0.95     16415
   macro avg       0.95      0.95      0.95     16415
weighted avg       0.95      0.95      0.95     16415



In [36]:
## Gradient Boosting Classifier
True_Negative, False_Positive, False_Negative, True_Positive = result(y_test , pred)
ACC = (True_Positive+True_Negative)/(True_Positive+False_Positive+False_Negative+True_Negative)*100
print(' Gradient Boosting Classifier')
print(f'Accuracy :{ACC}')
print(f'True Negattive: {(True_Negative/(True_Negative+False_Positive))*100}')
print(f'False Positive : {(False_Positive/(False_Positive+True_Negative))*100}')
print(f'False Negative : {(False_Negative/(True_Positive+False_Negative))*100}')
print(f'True Positive: {(True_Positive/(True_Positive+False_Negative))*100}')

 Gradient Boosting Classifier
Accuracy :94.95583307950047
True Negattive: 97.23371556921622
False Positive : 2.766284430783781
False Negative : 7.46606334841629
True Positive: 92.53393665158372


## Random Forest Classifier

In [37]:
from sklearn.ensemble import RandomForestClassifier


In [38]:
RFC_model = RandomForestClassifier(random_state = 0)
RFC_model.fit(x , y_train)
pred = RFC_model.predict(Xtest)
print("Random Forest Classifier :" , RFC_model.score(Xtest , y_test))
print(classification_report(y_test , pred))

Random Forest Classifier : 0.9285409686262565
              precision    recall  f1-score   support

        fake       0.91      0.95      0.93      8459
        real       0.95      0.90      0.92      7956

    accuracy                           0.93     16415
   macro avg       0.93      0.93      0.93     16415
weighted avg       0.93      0.93      0.93     16415



In [39]:
# Random Forest Classifier
True_Negative, False_Positive, False_Negative, True_Positive = result(y_test , pred)
ACC = (True_Positive+True_Negative)/(True_Positive+False_Positive+False_Negative+True_Negative)*100
print('Random Forest Classifier')
print(f'Accuracy :{ACC}')
print(f'True Negattive: {(True_Negative/(True_Negative+False_Positive))*100}')
print(f'False Positive : {(False_Positive/(False_Positive+True_Negative))*100}')
print(f'False Negative : {(False_Negative/(True_Positive+False_Negative))*100}')
print(f'True Positive: {(True_Positive/(True_Positive+False_Negative))*100}')

Random Forest Classifier
Accuracy :92.85409686262565
True Negattive: 95.28313039366355
False Positive : 4.716869606336446
False Negative : 9.728506787330318
True Positive: 90.27149321266968


In [40]:
import pickle
pickle.dump(PAC_model,open('Model.sav' , 'wb'))
pickle.dump(vectorizer ,open('vector','wb'))

In [41]:
pickle.dump(RFC_model,open('RFC_Model.sav' , 'wb'))
pickle.dump(LR_model,open('LR_Model.sav' , 'wb'))
pickle.dump(GBC_model,open('GBC_Model.sav' , 'wb'))
pickle.dump(DTC_model,open('DTC_Model.sav' , 'wb'))